# Webscraping de Inmobiliarias en Uruguay

Este notebook realiza scraping en páginas inmobiliarias reales de Uruguay para obtener propiedades en alquiler en Montevideo.

Se utilizan las páginas **InfoCasas** y **Gallito.com.uy** como ejemplo.

In [ ]:

import requests
from bs4 import BeautifulSoup
import json
import time


In [ ]:

def scrape_infocasas_montevideo(max_props=10):
    url = "https://www.infocasas.com.uy/alquiler/casas/montevideo"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    
    propiedades = []
    cards = soup.select("div.listing-item")[:max_props]  # Ajustar clase según HTML real
    for card in cards:
        try:
            link_tag = card.select_one("a[href]")
            link = link_tag["href"]
            if not link.startswith("http"):
                link = "https://www.infocasas.com.uy" + link
            
            precio = card.select_one(".first-price").get_text(strip=True) if card.select_one(".first-price") else None
            tamano = card.select_one(".property-size").get_text(strip=True) if card.select_one(".property-size") else None
            habitaciones = card.select_one(".rooms").get_text(strip=True) if card.select_one(".rooms") else None
            
            propiedades.append({
                "precio": precio,
                "tamano": tamano,
                "habitaciones": habitaciones,
                "link": link
            })
        except Exception as e:
            print("Error Infocasas:", e)
    return propiedades


In [ ]:

def scrape_gallito_montevideo(max_props=10):
    url = "https://www.gallito.com.uy/inmuebles/casas/alquiler/montevideo"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    
    propiedades = []
    cards = soup.select("div.card-inmueble")[:max_props]  # Ajustar clase según HTML real
    for card in cards:
        try:
            link_tag = card.select_one("a[href]")
            link = link_tag["href"]
            if not link.startswith("http"):
                link = "https://www.gallito.com.uy" + link
            
            precio = card.select_one(".precio").get_text(strip=True) if card.select_one(".precio") else None
            tamano = card.select_one(".m2").get_text(strip=True) if card.select_one(".m2") else None
            habitaciones = card.select_one(".habitaciones").get_text(strip=True) if card.select_one(".habitaciones") else None
            
            propiedades.append({
                "precio": precio,
                "tamano": tamano,
                "habitaciones": habitaciones,
                "link": link
            })
        except Exception as e:
            print("Error Gallito:", e)
    return propiedades


In [ ]:

def main():
    data = {"ciudades": []}
    
    ciudades = [
        {"nombre": "Montevideo - InfoCasas", "func": scrape_infocasas_montevideo},
        {"nombre": "Montevideo - Gallito", "func": scrape_gallito_montevideo}
    ]
    
    for c in ciudades:
        print(f"Scrapeando {c['nombre']} …")
        props = c["func"](max_props=10)
        data["ciudades"].append({"nombre": c["nombre"], "propiedades": props})
        time.sleep(2)
    
    # Guardar JSON
    with open("propiedades_inmobiliarias.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print("Datos guardados en propiedades_inmobiliarias.json")

if __name__ == "__main__":
    main()
